In [1]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_validate,KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [3]:
def merge_by_concat(df1, df2, merge_on):
  merged_gf = df1[merge_on]
  merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
  new_columns = [col for col in list(merged_gf) if col not in merge_on]
  df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
  return df1

In [4]:
train_values = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')
test_values = pd.read_csv('test_values.csv',index_col='building_id')

In [5]:
set_train = train_values.merge(train_labels)
set_train.drop( columns=["building_id"], inplace=True)

In [6]:
prob = pd.read_csv('probabilidades.csv')
probTest = pd.read_csv('probabilidadesTest.csv')

In [7]:
set_train['prob1_geo1'] = prob['prob1_geo1']
set_train['prob2_geo1'] = prob['prob2_geo1']
set_train['prob3_geo1'] = prob['prob3_geo1']
set_train['prob1_geo2'] = prob['prob1_geo2']
set_train['prob2_geo2'] = prob['prob2_geo2']
set_train['prob3_geo2'] = prob['prob3_geo2']
set_train['prob1_geo3'] = prob['prob1_geo3']
set_train['prob2_geo3'] = prob['prob2_geo3']
set_train['prob3_geo3'] = prob['prob3_geo3']


lista0 = probTest['prob1_geo1'].values.tolist()
lista1 = probTest['prob2_geo1'].values.tolist()
lista2 = probTest['prob3_geo1'].values.tolist()
lista3 = probTest['prob1_geo2'].values.tolist()
lista4 = probTest['prob2_geo2'].values.tolist()
lista5 = probTest['prob3_geo2'].values.tolist()
lista6 = probTest['prob1_geo3'].values.tolist()
lista7 = probTest['prob2_geo3'].values.tolist()
lista8 = probTest['prob3_geo3'].values.tolist()

In [8]:
test_values['prob1_geo1'] = lista0
test_values['prob2_geo1'] = lista1
test_values['prob3_geo1'] = lista2
test_values['prob1_geo2'] = lista3
test_values['prob2_geo2'] = lista4
test_values['prob3_geo2'] = lista5
test_values['prob1_geo3'] = lista6
test_values['prob2_geo3'] = lista7
test_values['prob3_geo3'] = lista8

In [9]:
df_full = pd.concat([set_train, test_values], axis=0).reset_index(drop=True)

In [10]:
geo_age_mean = df_full.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['age'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_age_mean'})
geo_height_mean = df_full.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['height_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_height_mean'})
geo_area_mean = df_full.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['area_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_area_mean'})


type_age_mean = df_full.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['age'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_age_mean'})
type_height_mean = df_full.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['height_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_height_mean'})
type_area_mean = df_full.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['area_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_area_mean'})


df_full = merge_by_concat(df_full, geo_age_mean, ['geo_level_1_id','geo_level_2_id','geo_level_3_id'])
df_full = merge_by_concat(df_full, geo_height_mean, ['geo_level_1_id','geo_level_2_id','geo_level_3_id'])
df_full = merge_by_concat(df_full, geo_area_mean, ['geo_level_1_id','geo_level_2_id','geo_level_3_id'])
df_full = merge_by_concat(df_full, type_age_mean, ['foundation_type','roof_type','ground_floor_type','other_floor_type'])
df_full = merge_by_concat(df_full, type_height_mean, ['foundation_type','roof_type','ground_floor_type','other_floor_type'])
df_full = merge_by_concat(df_full, type_area_mean, ['foundation_type','roof_type','ground_floor_type','other_floor_type'])


"""geo_age_mean2 = test_values.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['age'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_age_mean2'})
geo_height_mean2 = test_values.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['height_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_height_mean2'})
geo_area_mean2 = test_values.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['area_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_area_mean2'})

type_age_mean2 = test_values.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['age'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_age_mean2'})
type_height_mean2 = test_values.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['height_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_height_mean2'})
type_area_mean2 = test_values.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['area_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_area_mean2'})

test_values = merge_by_concat(test_values, geo_age_mean2, ['geo_level_1_id','geo_level_2_id','geo_level_3_id'])
test_values = merge_by_concat(test_values, geo_height_mean2, ['geo_level_1_id','geo_level_2_id','geo_level_3_id'])
test_values = merge_by_concat(test_values, geo_area_mean2, ['geo_level_1_id','geo_level_2_id','geo_level_3_id'])

test_values = merge_by_concat(test_values, type_age_mean2, ['foundation_type','roof_type','ground_floor_type','other_floor_type'])
test_values = merge_by_concat(test_values, type_height_mean2, ['foundation_type','roof_type','ground_floor_type','other_floor_type'])
test_values = merge_by_concat(test_values, type_area_mean2, ['foundation_type','roof_type','ground_floor_type','other_floor_type'])"""


"geo_age_mean2 = test_values.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['age'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_age_mean2'})\ngeo_height_mean2 = test_values.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['height_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_height_mean2'})\ngeo_area_mean2 = test_values.groupby(['geo_level_1_id','geo_level_2_id','geo_level_3_id'])['area_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'geo_grp_area_mean2'})\n\ntype_age_mean2 = test_values.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['age'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_age_mean2'})\ntype_height_mean2 = test_values.groupby(['foundation_type','roof_type','ground_floor_type','other_floor_type'])['height_percentage'].agg(['mean']).reset_index().rename(columns={'mean':'type_grp_height_mean2'})\ntype_area_mean2 = test_values.groupby(['foundati

In [11]:
set_train = df_full.iloc[0:260601, :]

In [12]:
train_values

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,0,0,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,0,0,0,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_other,prob1_geo1,prob2_geo1,prob3_geo1,prob1_geo2,prob2_geo2,prob3_geo2,prob1_geo3,prob2_geo3,prob3_geo3
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0.013066,0.179388,0.807546,0.007194,0.280576,0.712230,0.052632,0.263158,0.684211
99355,6,141,11987,2,25,13,5,t,r,n,...,0,0.086461,0.665354,0.248185,0.000000,0.819149,0.180851,0.000000,1.000000,0.000000
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0.129718,0.739603,0.130678,0.000000,0.825000,0.175000,0.000000,0.000000,1.000000
745817,26,39,633,1,0,19,3,t,r,x,...,0,0.354986,0.559142,0.085872,0.507429,0.452947,0.039624,0.571429,0.419355,0.009217
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0.013066,0.179388,0.807546,0.005181,0.300518,0.694301,0.000000,0.176471,0.823529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,0,0.035763,0.766337,0.197900,0.056604,0.764151,0.179245,0.194444,0.750000,0.055556
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0.054848,0.548349,0.396802,0.060519,0.541787,0.397695,0.000000,0.277778,0.722222
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0.129718,0.739603,0.130678,0.083744,0.812808,0.103448,0.090909,0.909091,0.000000


In [14]:
test_values = df_full.iloc[260601:347469, :]

In [15]:
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,prob3_geo2,prob1_geo3,prob2_geo3,prob3_geo3,geo_grp_age_mean,geo_grp_height_mean,geo_grp_area_mean,type_grp_age_mean,type_grp_height_mean,type_grp_area_mean
260601,17,596,11307,3,20,7,6,t,r,n,...,0.712230,0.052632,0.263158,0.684211,26.304348,6.000000,8.347826,30.543526,5.682254,7.515108
260602,6,141,11987,2,25,13,5,t,r,n,...,0.180851,0.000000,1.000000,0.000000,34.166667,4.750000,7.916667,30.543526,5.682254,7.515108
260603,22,19,10044,2,5,4,5,t,r,n,...,0.175000,0.000000,0.000000,1.000000,15.000000,4.333333,6.833333,30.543526,5.682254,7.515108
260604,26,39,633,1,0,19,3,t,r,x,...,0.039624,0.571429,0.419355,0.009217,12.681332,5.994055,20.692033,10.268362,3.120056,14.217514
260605,17,289,7970,3,15,8,7,t,r,q,...,0.694301,0.000000,0.176471,0.823529,45.000000,6.500000,7.791667,30.557604,5.803995,7.542252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347464,4,605,3623,3,70,20,6,t,r,q,...,0.179245,0.194444,0.750000,0.055556,41.521739,5.478261,15.456522,30.557604,5.803995,7.542252
347465,10,1407,11907,3,25,6,7,n,r,n,...,0.397695,0.000000,0.277778,0.722222,25.400000,5.920000,5.480000,30.543526,5.682254,7.515108
347466,22,1136,7712,1,50,3,3,t,r,n,...,0.103448,0.090909,0.909091,0.000000,32.666667,5.133333,8.733333,23.519524,2.999162,6.286473
347467,6,1041,912,2,5,9,5,t,r,n,...,0.361371,0.090909,0.681818,0.227273,18.265306,5.183673,7.887755,30.543526,5.682254,7.515108


In [16]:
"""#this block of code calculates the probabilities
damage1 = {}
damage2 = {}
damage3 = {}

for i,j in (set_train['geo_level_1_id'].value_counts()).iteritems():
    
    n1 = len(set_train[set_train['damage_grade']==1][set_train['geo_level_1_id']==i])
    n2 = len(set_train[set_train['damage_grade']==2][set_train['geo_level_1_id']==i])
    n3 = len(set_train[set_train['damage_grade']==3][set_train['geo_level_1_id']==i])

    damage1[i] = n1/j
    damage2[i] = n2/j
    damage3[i] = n3/j
    
# this block of code appends a new column/feature of probabilities
list1 = []
list2 = []
list3 = []

for i in set_train['geo_level_1_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

set_train['prob1_geo1'] = list1
set_train['prob2_geo1'] = list2
set_train['prob3_geo1'] = list3


list1 = []
list2 = []
list3 = []

for i in test_values['geo_level_1_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

test_values['prob1_geo1'] = list1
test_values['prob2_geo1'] = list2
test_values['prob3_geo1'] = list3"""


"#this block of code calculates the probabilities\ndamage1 = {}\ndamage2 = {}\ndamage3 = {}\n\nfor i,j in (set_train['geo_level_1_id'].value_counts()).iteritems():\n    \n    n1 = len(set_train[set_train['damage_grade']==1][set_train['geo_level_1_id']==i])\n    n2 = len(set_train[set_train['damage_grade']==2][set_train['geo_level_1_id']==i])\n    n3 = len(set_train[set_train['damage_grade']==3][set_train['geo_level_1_id']==i])\n\n    damage1[i] = n1/j\n    damage2[i] = n2/j\n    damage3[i] = n3/j\n    \n# this block of code appends a new column/feature of probabilities\nlist1 = []\nlist2 = []\nlist3 = []\n\nfor i in set_train['geo_level_1_id']:\n    list1.append(damage1.get(i))\n    list2.append(damage2.get(i))\n    list3.append(damage3.get(i))\n\nset_train['prob1_geo1'] = list1\nset_train['prob2_geo1'] = list2\nset_train['prob3_geo1'] = list3\n\n\nlist1 = []\nlist2 = []\nlist3 = []\n\nfor i in test_values['geo_level_1_id']:\n    list1.append(damage1.get(i))\n    list2.append(damage2.g

In [17]:
"""#this block of code calculates the probabilities
damage1 = {}
damage2 = {}
damage3 = {}

for i,j in ((set_train['geo_level_2_id'].value_counts()).iteritems()):
    
    n1 = len(set_train[set_train['damage_grade']==1][set_train['geo_level_2_id']==i])
    n2 = len(set_train[set_train['damage_grade']==2][set_train['geo_level_2_id']==i])
    n3 = len(set_train[set_train['damage_grade']==3][set_train['geo_level_2_id']==i])

    damage1[i] = n1/j
    damage2[i] = n2/j
    damage3[i] = n3/j
    

list1 = []
list2 = []
list3 = []

for i in set_train['geo_level_2_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

set_train['prob1_geo2'] = list1
set_train['prob2_geo2'] = list2
set_train['prob3_geo2'] = list3



list1 = []
list2 = []
list3 = []

for i in test_values['geo_level_2_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

test_values['prob1_geo2'] = list1
test_values['prob2_geo2'] = list2
test_values['prob3_geo2'] = list3"""


"#this block of code calculates the probabilities\ndamage1 = {}\ndamage2 = {}\ndamage3 = {}\n\nfor i,j in ((set_train['geo_level_2_id'].value_counts()).iteritems()):\n    \n    n1 = len(set_train[set_train['damage_grade']==1][set_train['geo_level_2_id']==i])\n    n2 = len(set_train[set_train['damage_grade']==2][set_train['geo_level_2_id']==i])\n    n3 = len(set_train[set_train['damage_grade']==3][set_train['geo_level_2_id']==i])\n\n    damage1[i] = n1/j\n    damage2[i] = n2/j\n    damage3[i] = n3/j\n    \n\nlist1 = []\nlist2 = []\nlist3 = []\n\nfor i in set_train['geo_level_2_id']:\n    list1.append(damage1.get(i))\n    list2.append(damage2.get(i))\n    list3.append(damage3.get(i))\n\nset_train['prob1_geo2'] = list1\nset_train['prob2_geo2'] = list2\nset_train['prob3_geo2'] = list3\n\n\n\nlist1 = []\nlist2 = []\nlist3 = []\n\nfor i in test_values['geo_level_2_id']:\n    list1.append(damage1.get(i))\n    list2.append(damage2.get(i))\n    list3.append(damage3.get(i))\n\ntest_values['prob1

In [18]:
"""damage1 = {}
damage2 = {}
damage3 = {}

for i,j in ((set_train['geo_level_3_id'].value_counts()).iteritems()):
    
    n1 = len(set_train[set_train['damage_grade']==1][set_train['geo_level_3_id']==i])
    n2 = len(set_train[set_train['damage_grade']==2][set_train['geo_level_3_id']==i])
    n3 = len(set_train[set_train['damage_grade']==3][set_train['geo_level_3_id']==i])

    damage1[i] = n1/j
    damage2[i] = n2/j
    damage3[i] = n3/j
    
list1 = []
list2 = []
list3 = []

for i in set_train['geo_level_3_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

set_train['prob1_geo3'] = list1
set_train['prob2_geo3'] = list2
set_train['prob3_geo3'] = list3


list1 = []
list2 = []
list3 = []

for i in test_values['geo_level_3_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

test_values['prob1_geo3'] = list1
test_values['prob2_geo3'] = list2
test_values['prob3_geo3'] = list3"""

"damage1 = {}\ndamage2 = {}\ndamage3 = {}\n\nfor i,j in ((set_train['geo_level_3_id'].value_counts()).iteritems()):\n    \n    n1 = len(set_train[set_train['damage_grade']==1][set_train['geo_level_3_id']==i])\n    n2 = len(set_train[set_train['damage_grade']==2][set_train['geo_level_3_id']==i])\n    n3 = len(set_train[set_train['damage_grade']==3][set_train['geo_level_3_id']==i])\n\n    damage1[i] = n1/j\n    damage2[i] = n2/j\n    damage3[i] = n3/j\n    \nlist1 = []\nlist2 = []\nlist3 = []\n\nfor i in set_train['geo_level_3_id']:\n    list1.append(damage1.get(i))\n    list2.append(damage2.get(i))\n    list3.append(damage3.get(i))\n\nset_train['prob1_geo3'] = list1\nset_train['prob2_geo3'] = list2\nset_train['prob3_geo3'] = list3\n\n\nlist1 = []\nlist2 = []\nlist3 = []\n\nfor i in test_values['geo_level_3_id']:\n    list1.append(damage1.get(i))\n    list2.append(damage2.get(i))\n    list3.append(damage3.get(i))\n\ntest_values['prob1_geo3'] = list1\ntest_values['prob2_geo3'] = list2\nte

In [19]:
def encoding(values, columnData, setDatos):
    df = pd.DataFrame()
    column = columnData
    if(setDatos=='set_train'):
        df[values] = pd.get_dummies(set_train[column])
    else:
        df[values] = pd.get_dummies(test_values[column])
    df['sumatory'] = 0
    for i in range(len(values)):
        df['sumatory'] += df[values[i]] * df[values[i]].mean()
    
    return df['sumatory']

In [20]:
columnas_seleccionadas = ['age',
                         'geo_level_1_id',
                         'geo_level_2_id',
                         'geo_level_3_id',
                         'count_families',
                         'base_condition',
                         #'prob1_geo1', 'prob2_geo1', 'prob3_geo1', 
                         #'prob1_geo2', 'prob2_geo2', 'prob3_geo2', 
                        #'prob1_geo3', 'prob2_geo3', 'prob3_geo3', 
                         'volume_percentage', #'promedio',
                         'area_percentage',
                         'height_percentage', 
                          'geo_grp_age_mean',
                          'geo_grp_height_mean',
                          'geo_grp_area_mean',
                          'type_grp_age_mean',
                          'type_grp_height_mean',
                          'type_grp_area_mean',
                         'count_floors_pre_eq',
                         'has_secondary_use','has_secondary_use_agriculture', 
                         'has_secondary_use_hotel', 'has_secondary_use_rental',
                         'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_use_industry', 
                         'has_secondary_use_health_post', 'has_secondary_use_gov_office','has_secondary_use_use_police',
                         'has_secondary_use_other', 
                         'has_superstructure_rc_engineered', 'has_superstructure_rc_non_engineered', 'has_superstructure_cement_mortar_brick',
                         'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_mud_mortar_stone',
                         'has_superstructure_stone_flag', 'has_superstructure_timber', 'has_superstructure_adobe_mud', 
                         'has_superstructure_bamboo', 'has_superstructure_other',
                         'land_surface_condition_n', 'land_surface_condition_o', 'land_surface_condition_t',
                         'foundation_type_h', 'foundation_type_i', 'foundation_type_r', 'foundation_type_u', 'foundation_type_w',
                         'roof_type_n', 'roof_type_q', 'roof_type_x',
                         'ground_floor_type_f', 'ground_floor_type_m', 'ground_floor_type_v', 'ground_floor_type_x', 'ground_floor_type_z',
                         'other_floor_type_j', 'other_floor_type_q', 'other_floor_type_s', 'other_floor_type_x',
                         'position_j', 'position_o', 'position_s', 'position_t',
                       #  'legal_ownership_status_a', 'legal_ownership_status_r', 'legal_ownership_status_v', 'legal_ownership_status_w',
                         'plan_configuration_a', 'plan_configuration_c', 'plan_configuration_d', 'plan_configuration_f', 'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o', 'plan_configuration_q', 'plan_configuration_s', 'plan_configuration_u',
                         'damage_grade'
                         ]

In [21]:
set_train['promedio'] = set_train['height_percentage'] / set_train['count_floors_pre_eq']
test_values['promedio'] = test_values['height_percentage'] / test_values['count_floors_pre_eq']
#set_train['age'] = (set_train['age'] + 1).apply(np.log)
set_train['base_condition'] = encoding(['h', 'i', 'r', 'u', 'w'] , 'foundation_type', 'set_train') + encoding(['n', 'o', 't'] , 'land_surface_condition', 'set_train')
test_values['base_condition'] = encoding(['h', 'i', 'r', 'u', 'w'] , 'foundation_type', 'test_values') + encoding(['n', 'o', 't'] , 'land_surface_condition', 'test_values')

set_train['volume_percentage'] = set_train['area_percentage'] * set_train['height_percentage']
test_values['volume_percentage'] = test_values['area_percentage'] * test_values['height_percentage']

In [22]:
set_train['roof_type'] = encoding(['n', 'q', 'x'] , 'roof_type', 'set_train')
set_train['ground_floor_type'] = encoding(['f','m','v','x','z'] , 'ground_floor_type', 'set_train')
set_train['other_floor_type'] = encoding(['j','q','s','x'] , 'other_floor_type', 'set_train')
set_train['land_surface_condition'] = encoding(['n','o','t'],  'land_surface_condition', 'set_train')
set_train['foundation_type'] = encoding(['h','i','r','u','w'], 'foundation_type', 'set_train')                                               
set_train['position'] = encoding(['j','o','s','t'], 'position', 'set_train')                                              
set_train['plan_configuration'] = encoding(['a', 'c', 'd', 'f', 'm', 'n', 'o', 'q', 's', 'u'], 'plan_configuration', 'set_train')

test_values['roof_type'] = encoding(['n', 'q', 'x'] , 'roof_type', 'test_values')
test_values['ground_floor_type'] = encoding(['f','m','v','x','z'] , 'ground_floor_type', 'test_values')
test_values['other_floor_type'] = encoding(['j','q','s','x'] , 'other_floor_type', 'test_values')
test_values['land_surface_condition'] = encoding(['n','o','t'],  'land_surface_condition', 'test_values')
test_values['foundation_type'] = encoding(['h','i','r','u','w'], 'foundation_type', 'test_values')                                               
test_values['position'] = encoding(['j','o','s','t'], 'position', 'test_values')                                              
test_values['plan_configuration'] = encoding(['a', 'c', 'd', 'f', 'm', 'n', 'o', 'q', 's', 'u'], 'plan_configuration', 'test_values')

In [23]:
#set_train = set_train.loc[set_train['count_floors_pre_eq'] <= 5]
#set_train = set_train.loc[set_train['age'] <= 100]

In [24]:
data_name = ['has_superstructure_rc_engineered', 'has_superstructure_rc_non_engineered', 'has_superstructure_cement_mortar_brick',
'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_mud_mortar_stone',
'has_superstructure_stone_flag', 'has_superstructure_timber', 'has_superstructure_adobe_mud', 
'has_superstructure_bamboo', 'has_superstructure_other']

data_names = ['has_superstructure_stone_flag', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
'has_superstructure_mud_mortar_brick',
'has_superstructure_timber', 'has_superstructure_bamboo',
'has_superstructure_other', 'has_superstructure_cement_mortar_stone',
'has_superstructure_rc_non_engineered', 'has_superstructure_cement_mortar_brick' ,
'has_superstructure_rc_engineered']
data_values = [2.4526045159847976,
 2.346841581157726,
 2.337918078939549,
 2.271334422351152,
 2.165229430975063,
 2.111717974180735,
 2.088348271446863,
 1.9675721204464098,
 1.7791106701542347,
 1.6929632086543858,
 1.3746964545896065]


data_value = [0.05927992, 0.01710788, 0.08324661, 0.00788696, 0.0638134 ,
       0.6152512 , 0.03090751, 0.02455271, 0.07571122, 0.01783531,
       0.00440734]

for i in range(11):
    set_train.loc[:, [data_names[i]]] = set_train.loc[:, [data_names[i]]]*data_values[i]
set_train['resistance'] = 0
for x in range(11):
    set_train['resistance'] += set_train[data_names[x]]

for i in range(11):
    test_values.loc[:, [data_names[i]]] = test_values.loc[:, [data_names[i]]]*data_values[i]
test_values['resistance'] = 0
for x in range(11):
    test_values['resistance'] += test_values[data_names[x]]

In [25]:
set_train[['land_surface_condition_n', 'land_surface_condition_o', 'land_surface_condition_t']] = pd.get_dummies(set_train['land_surface_condition'])
set_train[['foundation_type_h', 'foundation_type_i', 'foundation_type_r', 'foundation_type_u', 'foundation_type_w']] = pd.get_dummies(set_train['foundation_type'])
set_train[['roof_type_n', 'roof_type_q', 'roof_type_x']] = pd.get_dummies(set_train['roof_type'])
set_train[['ground_floor_type_f', 'ground_floor_type_m', 'ground_floor_type_v', 'ground_floor_type_x', 'ground_floor_type_z']] = pd.get_dummies(set_train['ground_floor_type'])
set_train[['other_floor_type_j', 'other_floor_type_q', 'other_floor_type_s', 'other_floor_type_x']] = pd.get_dummies(set_train['other_floor_type'])
set_train[['position_j', 'position_o', 'position_s', 'position_t']] = pd.get_dummies(set_train['position'])
set_train[['plan_configuration_a', 'plan_configuration_c', 'plan_configuration_d', 'plan_configuration_f', 'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o', 'plan_configuration_q', 'plan_configuration_s', 'plan_configuration_u']] = pd.get_dummies(set_train['plan_configuration'])


test_values[['land_surface_condition_n', 'land_surface_condition_o', 'land_surface_condition_t']] = pd.get_dummies(test_values['land_surface_condition'])
test_values[['foundation_type_h', 'foundation_type_i', 'foundation_type_r', 'foundation_type_u', 'foundation_type_w']] = pd.get_dummies(test_values['foundation_type'])
test_values[['roof_type_n', 'roof_type_q', 'roof_type_x']] = pd.get_dummies(test_values['roof_type'])
test_values[['ground_floor_type_f', 'ground_floor_type_m', 'ground_floor_type_v', 'ground_floor_type_x', 'ground_floor_type_z']] = pd.get_dummies(test_values['ground_floor_type'])
test_values[['other_floor_type_j', 'other_floor_type_q', 'other_floor_type_s', 'other_floor_type_x']] = pd.get_dummies(test_values['other_floor_type'])
test_values[['position_j', 'position_o', 'position_s', 'position_t']] = pd.get_dummies(test_values['position'])
test_values[['plan_configuration_a', 'plan_configuration_c', 'plan_configuration_d', 'plan_configuration_f', 'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o', 'plan_configuration_q', 'plan_configuration_s', 'plan_configuration_u']] = pd.get_dummies(test_values['plan_configuration'])

In [26]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [27]:
train, test = train_test_split(set_train[columnas_seleccionadas], test_size = 0.20, shuffle = False)

In [28]:
train_values_subset = train.iloc[:, 0:-1]
train_labels = train.loc[:, 'damage_grade']

In [29]:
test_values_subset = test.iloc[:, 0:-1]
test_labels = test.loc[:, 'damage_grade']

In [30]:
rf_model = XGBClassifier(n_jobs=-1,n_estimators= 400, max_depth= 10,learning_rate= 0.1, eval_metric='mlogloss', colsample_bytree = 0.9)    
train_values_subset = pd.get_dummies(train_values_subset)

In [31]:
start_time = timer(None)
rf_model.fit(train_values_subset, train_labels.values.ravel())
timer(start_time) 


 Time taken: 0 hours 7 minutes and 56.07 seconds.


In [32]:
preds = rf_model.predict(test_values_subset)
f1_score(test_labels, preds, average='micro')
# 0.747025637089346 con count floors <= 6
# SIN count floors 0.7474530419600546

# 0.7606530956812033 --- Submit 0.7445 usando Volume, y sacando geolevel 

# 0.7602310009401202 --- Submit 0.7442  usando VOLUME - Promedio - COUNT FLOORS

# 0.7614205406649911 --- Submit 0.7455  usando VOLUME 
#                        SUBMIT 0.7455  usando VOLUME - BASE CONDITION
# 0.7602310009401202 --- Submit 0.7452  usando VOLUME - PROMEDIO
# 0.7604420483106616 --- Submit 0.7453  usando VOLUME - PROMEDIO  -                  SIN COUNT FLOORS/HEIGHT
# 0.7603077454384988 --- Submit 0.7450  usando VOLUME - PROMEDIO  - BASE CONDITION - SIN COUNT FLOORS/HEIGHT

# 0.7609600736747184 --- Submit 0.7453  usando VOLUME - PROMEDIO CON HEIGHT y SIN count floors

# 0.7610751904222866 --- Submit 0.7447 usando VOLUME - PROMEDIO - BASE CONDITION - SIN height/count y SIN roof-ground-other plan configuration

# 0.7603077454384988 SIN plan configuration VOLUME 0.7452 <<-<-<-<-<-< submit 0.7452

# 0.7605187928090407 A PROBAR - VOLUME BASE CONDITION sin land surface - foundation type 0.7453
# 0.7612670516682335 A PROBAR - VOLUME BASE CONDITION sin encodear land surface found. position y plan configuration previamente 0.7453
# 0.760614723432014  A PROBAR - VOLUME BASE CONDITION sin encodear position y plan configuration  0.7453+


# 0.7500815410295274     SUBMIT MAXIMO 0.7479       CON VOLUME - BASE CONDITION 
# 0.7515205003741295     SUBMIT 0.7472          VOLUME sin base condition 




# 0.7497xxxx con volume, base condition, PROMEDIO y SIN height

0.7500815410295274

In [33]:
test_values_subset = test_values[columnas_seleccionadas[0:-1]]
test_values_subset

,age,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_families,base_condition,volume_percentage,area_percentage,height_percentage,geo_grp_age_mean,...,plan_configuration_a,plan_configuration_c,plan_configuration_d,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u
260601,20,17,596,11307,1,1.673320,42,7,6,26.304348,...,0,0,0,0,0,0,0,0,0,1
260602,25,6,141,11987,1,1.673320,65,13,5,34.166667,...,0,0,0,0,0,0,0,0,0,1
260603,5,22,19,10044,1,1.673320,20,4,5,15.000000,...,0,0,0,0,0,0,0,0,0,1
260604,0,26,39,633,2,1.673320,57,19,3,12.681332,...,0,0,0,0,0,0,0,0,0,1
260605,15,17,289,7970,1,1.673320,56,8,7,45.000000,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347464,70,4,605,3623,1,1.673320,120,20,6,41.521739,...,0,0,0,0,0,0,0,0,0,1
347465,25,10,1407,11907,1,0.979221,42,6,7,25.400000,...,0,0,0,0,0,0,0,0,0,1
347466,50,22,1136,7712,1,1.673320,9,3,3,32.666667,...,0,0,0,0,0,0,0,0,0,1
347467,5,6,1041,912,1,1.673320,45,9,5,18.265306,...,0,0,0,0,0,0,0,0,0,1


In [34]:
predictions = rf_model.predict(test_values_subset)

In [35]:
c = []
for i in range(len(predictions)):
    c.append(int(predictions[i]))

In [36]:
submission_format = pd.read_csv('submission_format.csv',index_col='building_id')
my_submission = pd.DataFrame(data=c, columns=submission_format.columns,
                            index=submission_format.index)
my_submission.to_csv('submision.csv')

In [37]:
#copiaProb = pd.DataFrame()
#copiaProbTest = pd.DataFrame()

#copiaProb = set_train.loc[:, ['prob1_geo1', 'prob2_geo1', 'prob3_geo1', 
                        # 'prob1_geo2', 'prob2_geo2', 'prob3_geo2', 
                       #  'prob1_geo3', 'prob2_geo3', 'prob3_geo3']]
#copiaProbTest = test_values.loc[:, ['prob1_geo1', 'prob2_geo1', 'prob3_geo1', 
                      #   'prob1_geo2', 'prob2_geo2', 'prob3_geo2', 
                      #   'prob1_geo3', 'prob2_geo3', 'prob3_geo3']]

#copiaProb.to_csv('probabilidades.csv')
#copiaProbTest.to_csv('probabilidadesTest.csv')